<a href="https://colab.research.google.com/github/gommungommun/ResNet-18/blob/main/ResNet_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/gommungommun/ResNet-18.git

fatal: destination path 'ResNet-18' already exists and is not an empty directory.


In [4]:
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union

import torch
import torch.nn as nn
from torch import Tensor

##1. Residual Block

----

* in_planes: 입력 필터 개수

* out_planes: 출력 필터 개수

* groups: input과 output의 connection을 제어

* dilation: 커널 원소간의 거리


----

###1. __init__

처음 Normalization Layer가 없는 경우 nn.BatchNorm2d로 지정

###2. forward

identity 변수에 입력텐서 x를 저장

정의해둔 신경망을 거친 뒤 out 과 identity를 더한 후 relu를 거친다.

downsampling이 필요한 경우 이를 진행

In [5]:
# 원래 안하지만 편의를 위해 Type Hinting
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [6]:
class BasicBlock(nn.Module):
    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()

        # Normalization Layer
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # downsampling이 필요한 경우 downsample layer를 block에 인자로 넣어주어야함
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity  # residual connection
        out = self.relu(out)

        return out

##2. Bottleneck class

ResNet18 이므로 여기서 쓰이지는 않으나 50 이상 부터는 쓰임

---
###**[Bottleneck 을 따로 정의하는 이유]**

1. 계산 효율성
* 1x1컨볼루션을 사용해서 채널 수를 줄였다가 늘림
* 이를 통해 3x3컨볼루션의 계산량을 줄일 수 있음

2. 모델 크기에 따른 선택

3. 메모리 효율성
* 중간 레이어의 채널 수를 줄여 메모리 사용을 효율적으로 관리함

In [7]:
class Bottleneck(nn.Module):
    # 확장 계수 (일반적으로 4를 사용)
    expansion = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        # 1x1 컨볼루션
        self.conv1 = conv1x1(inplanes, planes)
        self.bn1 = norm_layer(planes)

        # 3x3 컨볼루션
        self.conv2 = conv3x3(planes, planes, stride, groups, dilation)
        self.bn2 = norm_layer(planes)

        # 1x1 컨볼루션 (채널 확장)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        # 1x1 컨볼루션
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        # 3x3 컨볼루션
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        # 1x1 컨볼루션 (채널 확장)
        out = self.conv3(out)
        out = self.bn3(out)

        # 다운샘플링이 필요한 경우
        if self.downsample is not None:
            identity = self.downsample(x)

        # 스킵 커넥션 더하기
        out += identity
        out = self.relu(out)

        return out

##3. ResNet class

###1. init

Normalization Layer가 없는 경우에 생성

inplaens, dilation, groups는 각각 64, 1, 1로 고정한다


----

###2. _make_layer

residual block을 쌓는다. 필터의 개수는 각 block들을 거치면서 2배씩 늘어나게 된다.

모든 block을 거친 후 Adaptive AvgPool2d를 이용해 (n, 512, 1, 1)의 tensor로 만듦

그 다음 fc layer를 연결하면 끝임

* block: BasicBlock 구조를 사용

* plane: input shape

* blocks: layer를 반복해서 쌓는 개수

* stride, dilate: 고정

중간에 downsampling layer를 생성하는 이유는 stride가 1이 아니기 때문에 크기가 줄어들 경우 혹은 plane의 크기가 맞지 않을때 downsampling을 해야하기 때문임

----
###3. Forward

텐서의 사이즈 변화를 나타내기 위해 레이어 별로 사이즈를 출력하도록 함


In [8]:
class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        dropout_rate: float = 0.5
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer  # batch norm layer

        self.inplanes = 64  # input shape
        self.dilation = 1  # dilation fixed
        self.groups = 1  # groups fixed

        # input block
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # residual blocks
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=False)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=False)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=False)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fc = nn.Linear(512, num_classes)


        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None

        # downsampling 필요할경우 downsample layer 생성
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes, stride),
                norm_layer(planes),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.dilation, norm_layer))
        self.inplanes = planes
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                 dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x: Tensor) -> Tensor:
        print('input shape:', x.shape)
        x = self.conv1(x)
        print('conv1 shape:', x.shape)
        x = self.bn1(x)
        print('bn1 shape:', x.shape)
        x = self.relu(x)
        print('relu shape:', x.shape)
        x = self.maxpool(x)
        print('maxpool shape:', x.shape)

        x = self.layer1(x)
        print('layer1 shape:', x.shape)
        x = self.layer2(x)
        print('layer2 shape:', x.shape)
        x = self.layer3(x)
        print('layer3 shape:', x.shape)
        x = self.layer4(x)
        print('layer4 shape:', x.shape)

        x = self.avgpool(x)
        print('avgpool shape:', x.shape)
        x = torch.flatten(x, 1)
        print('flatten shape:', x.shape)
        x = self.dropout(x)
        print('Dropout shape: ', x.shape)
        x = self.fc(x)
        print('fc shape:', x.shape)

        return x

##4. 결과
    Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        3: 입력 채널 수 (RGB 이미지는 3채널)
        64: 출력 채널 수
        kernel_size=(7, 7): 합성곱 필터의 크기
        stride=(2, 2): 필터가 이동하는 간격
        padding=(3, 3): 입력 이미지 주변에 추가하는 패딩 크기
        bias=False: 편향 사용 여부


    BatchNorm2d(64, eps=1e-05, momentum=0.1)
        64: 입력 특성 맵의 채널 수
        eps: 수치 안정성을 위한 작은 상수
        momentum: 이동 평균을 계산할 때 사용되는 모멘텀 값


    ReLU(inplace=True)
        inplace=True: 입력을 받은 텐서를 직접 수정하여 메모리 효율성 향상


    MaxPool2d(kernel_size=3, stride=2, padding=1)
        kernel_size=3: 풀링 윈도우의 크기
        stride=2: 풀링 윈도우가 이동하는 간격
        padding=1: 패딩 크기


    BasicBlock 내부 구조:
        conv1, conv2: 3x3 컨볼루션 레이어
        bn1, bn2: 배치 정규화 레이어
        ReLU: 활성화 함수

    affine=True
        BatchNorm 레이어가 학습 가능한 감마(γ)와 베타(β) 파라미터를 사용할지 여부
        True일 경우: y = γ * x_normalized + β 형태로 변환 가능
        정규화된 데이터를 다시 스케일링하고 이동시킬 수 있는 유연성 제공
        모델의 표현력을 높이는 역할


    track_running_stats=True
        배치별 평균과 분산의 이동평균(running mean/variance)을 계산하고 저장할지 여부
        True일 경우:
        학습 중에는 배치별 통계를 사용하면서 이동평균을 업데이트
        추론(inference) 시에는 저장된 이동평균을 사용
        False일 경우:
        항상 현재 배치의 통계만 사용
        배치 크기가 1일 때는 문제가 될 수 있음

In [12]:
model = ResNet(BasicBlock, [2, 2, 2, 2])
print(model)
# model_b = ResNet(Bottleneck, [3, 4, 6, 3])
# print(model_b)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
x = torch.randn(32, 3, 224, 224)
print('output shape: ', model(x).shape)

input shape: torch.Size([32, 3, 224, 224])
conv1 shape: torch.Size([32, 64, 112, 112])
bn1 shape: torch.Size([32, 64, 112, 112])
relu shape: torch.Size([32, 64, 112, 112])
maxpool shape: torch.Size([32, 64, 56, 56])
layer1 shape: torch.Size([32, 64, 56, 56])
layer2 shape: torch.Size([32, 128, 28, 28])
layer3 shape: torch.Size([32, 256, 14, 14])
layer4 shape: torch.Size([32, 512, 7, 7])
avgpool shape: torch.Size([32, 512, 1, 1])
flatten shape: torch.Size([32, 512])
Dropout shape:  torch.Size([32, 512])
fc shape: torch.Size([32, 1000])
output shape:  torch.Size([32, 1000])
